In [1]:
import pygame, sys, random
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

num_episodes = 1001
dis = 0.9
rList = []

obstacleRadius = 10
agentRadius = 5
goalRadius = 10
# get size of state and action from environment

boundaryPos = [50, 50]
boundaryLength = [200,200]
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2 + 20, boundaryPos[1] + boundaryLength[1] / 2 + 20]
initPosAgentTmp = initPosAgent
initPosGoal =  [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]

moveObstacles = False
action_size = 9
obsNumber = 4
state_size = 2
hidden_size = 128
obsAngleUnit = 1
hidden_size = 2
learning_rate = 0.00001

In [2]:
def initialize_variable(scope_name, var_name, shape):
    with tf.variable_scope(scope_name) as scope:
        v = tf.get_variable(var_name, shape)
        scope.reuse_variables()

def get_scope_variable(scope_name, var_name):
    with tf.variable_scope(scope_name, reuse=True):
        v = tf.get_variable(var_name)
    return v

In [3]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < boundaryPos[0]):
        xPos = boundaryPos[0]
#         xPos = boundaryPos[0] + boundaryLength[0]
        flagWall = -1
    elif(xPos > boundaryPos[0] + boundaryLength[0]):
        xPos = boundaryPos[0] + boundaryLength[0]
#         xPos = boundaryPos[0]
        flagWall = -1
    if(yPos < boundaryPos[1]):
        yPos = boundaryPos[1]
#         yPos = boundaryPos[1] + boundaryLength[1]
        flagWall = -1
    elif(yPos > boundaryPos[1] + boundaryLength[1]):
        yPos = boundaryPos[1] + boundaryLength[1]
#         yPos = boundaryPos[1]
        flagWall = -1
    return [int(round(xPos)), int(round(yPos)), flagWall]

In [4]:
def takeAction(action):
    xAction = 0
    yAction = 0
    actionOffset = 2
    if action == 0:
        xAction = actionOffset
    elif action == 1:
        xAction = actionOffset
        yAction = actionOffset
    elif action == 2:
        xAction = actionOffset
        yAction = -actionOffset
    elif action == 3:
        xAction = -actionOffset
        yAction = actionOffset
    elif action == 4:
        xAction = -actionOffset
    elif action == 5:
        xAction = -actionOffset
        yAction = -actionOffset
    elif action == 6:
        yAction = -actionOffset
    elif action == 7:
        yAction = actionOffset
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [5]:
def stateGenerator(intenPref, distPref):
    returnSum = [intenPref, distPref]
    returnSum = np.reshape(returnSum, [1, state_size])
#     print returnSum
    return returnSum

In [6]:
def rangeFinder(allObsPos, rangeCenter):
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
    index = np.argmin(allObsAgtDistance)
    return index

In [7]:
X = tf.placeholder(tf.float32,  [None, state_size], name = "input_x")
initialize_variable("scope1", "W1", [2, 1])
W1 = get_scope_variable("scope1", "W1")
initialize_variable("scope1", "W2", [1, 128])
layer1 = tf.nn.relu(tf.matmul(X, W1))
W2 = get_scope_variable("scope1", "W2")
initialize_variable("scope1", "W3", [128, 1])
layer2 = tf.nn.relu(tf.matmul(layer1, W2))
W3 = get_scope_variable("scope1", "W3")

Qpred = tf.matmul(layer2, W3)
Y = tf.placeholder(dtype = tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [8]:
#display
pygame.init()
screen = pygame.display.set_mode([300,300])
screen.fill([200, 200, 200])

episodeNo = []
scorePlot = []

obsAngleIdx = 0
# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
posOffset = 60
obstaclePos[0][0] = int(initPosGoal[0] + posOffset) 
obstaclePos[0][1] = int(initPosGoal[1]) 
obstaclePos[1][0] = int(initPosGoal[0]) 
obstaclePos[1][1] = int(initPosGoal[1] + posOffset) 
obstaclePos[2][0] = int(initPosGoal[0] - posOffset) 
obstaclePos[2][1] = int(initPosGoal[1]) 
obstaclePos[3][0] = int(initPosGoal[0]) 
obstaclePos[3][1] = int(initPosGoal[1] - posOffset) 
beta = 0.0
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    Qs = np.zeros(action_size)
#     Qs = [0 for _ in range(action_size)]

    for episode in range(num_episodes):
        done = False
        score = 0
        x = initPosAgent[0]
        y = initPosAgent[1]
        print("Episode ", episode, "Starts!")
#         goalRadius = initGoalRadius - episode / 25
        xMove = 0
        yMove = 0
        idx = rangeFinder(obstaclePos, [x,y])
        FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
        FeatureVec2 = 100000.0 / (beta + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
#         FeatureVec2 = (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2

        state = stateGenerator(FeatureVec1, FeatureVec2)
        e = 1.0 / ((episode/100.0) + 1.0)
#             e = 0.01
#         e = 1.0 / (1.0 + episode / 100.0)
        rAll = 0
        step_count = 0
        while not done:
            step_count += 1
            Qinput = np.reshape(state, [1, state_size])
            
            for i in range(0,action_size):
                [tmpXMove, tmpYMove] = takeAction(i)
                tmpX = x + tmpXMove
                tmpY = y + tmpYMove
                idx = rangeFinder(obstaclePos, [tmpX,tmpY])
                FeatureVec1 = (tmpX - initPosGoal[0])**2 + (tmpY - initPosGoal[1])**2
                FeatureVec2 = 100000.0 / (beta + (tmpX - obstaclePos[idx][0])**2 + (tmpY - obstaclePos[idx][1])**2)
#                 FeatureVec2 = (tmpX - obstaclePos[idx][0])**2 + (tmpY - obstaclePos[idx][1])**2
                tmpNextState = stateGenerator(FeatureVec1, FeatureVec2)
                QinputPred = np.reshape(tmpNextState, [1, state_size])
                Qs[i] = sess.run(Qpred, feed_dict = {X: QinputPred})
#             print Qs
#             print "========================================"
            if np.random.rand(1) < e:
                action = random.randrange(0,8)
            else:
                action = np.argmax(Qs)
#             print action
            xMove = 0
            yMove = 0
            [xMove, yMove] = takeAction(action)

            x = x + xMove
            y = y + yMove
            
            wallFlag = 0
            collisionFlag = 0
            [x, y, wallFlag] = ckWall(x, y)
            pygame.draw.circle(screen, [100,100,255],[x,y],agentRadius,0)
            idx = rangeFinder(obstaclePos, [x,y])
            FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
            FeatureVec2 = 100000.0 / (beta + (tmpX - obstaclePos[idx][0])**2 + (tmpY - obstaclePos[idx][1])**2)
#             FeatureVec2 = (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2
            next_state = stateGenerator(FeatureVec1, FeatureVec2)
            ## Draw Environment (Obstacle + Map)
            if(math.sqrt((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) <= goalRadius + agentRadius):
                print("Goal Reached!")
                reward = 2000.0
                collisionFlag = 1
                done = True
            for i in range(0,obsNumber):
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= obstacleRadius + agentRadius:
                    print("Collision!", idx)
                    reward = -15.0
                    collisionFlag = -1
                    done = True
                    break
            Qinput1 = np.reshape(next_state, [1, state_size])
            
            if not done:
                reward = -7.0 * math.sqrt((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) / 1000.0
#                 reward = 0
                QcurrPred = reward + dis * Qs[action]
            
            score += reward
            
            if done:
                episodeNo = episodeNo + [episode]
                if collisionFlag == 1:
                    QcurrPred = 2000.0
                    rList.append(1)
                elif collisionFlag == -1:
                    QcurrPred = -15.0
                    rList.append(0)
#             Qs1 = sess.run(Qpred, feed_dict = {X: Qinput1})
                print score
                scorePlot = scorePlot + [score]
#             QcurrPred = reward + dis * np.max(Qs)

            print QcurrPred, '\r',
            sess.run(train, feed_dict={X: Qinput1, Y: QcurrPred})
            state = next_state

            if done:
                if collisionFlag == 1:
                    if initPosAgent[0] >= boundaryPos[0] + boundaryLength[0] / 2 + 50 and initPosAgent[0] <= boundaryPos[0] + boundaryLength[0] / 2 + 99:
                        initPosAgent[1] = initPosAgent[1] - 1
                        initPosAgent[0] = initPosAgent[0] + 1
                    elif initPosAgent[0] < boundaryPos[0] + boundaryLength[0] / 2 + 50:
                        initPosAgent[0] = initPosAgent[0] + 1
                        initPosAgent[1] = initPosAgent[1] + 1
                
                for i in range(0,obsNumber):
                    pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                if episode % 100 == 0:
                    plt.plot(episodeNo, scorePlot, linewidth = 0.3)
                    plt.savefig("/home/howoongjun/RLPractice/Practice/DataSave/RewardPlot.png", dpi=300)
            pygame.draw.circle(screen, [100,255,100], initPosGoal, goalRadius, 1)
            pygame.draw.rect(screen, [255,100,100],[boundaryPos[0],boundaryPos[1], boundaryLength[0], boundaryLength[1]],1)
            pygame.display.flip()
            screen.fill([200,200,200])
#             print sess.run(loss, feed_dict = {X:Qinput, Y:QcurrPred}), '\r',
#             print sess.run(Qpred, feed_dict = {X: QinputPred}), '\r',
            
    print sess.run(W1)

('Episode ', 0, 'Starts!')
('Collision!', 3)
-630.399016411
('Episode ', 1, 'Starts!')
('Collision!', 0)
-795.418249066
('Episode ', 2, 'Starts!')
('Collision!', 1)
-115.973373039
('Episode ', 3, 'Starts!')
Goal Reached!63 
1910.11889904
('Episode ', 4, 'Starts!')
Goal Reached!49 
1921.70362003
('Episode ', 5, 'Starts!')
('Collision!', 0)
-41.7595657288
('Episode ', 6, 'Starts!')
Goal Reached!63  
1927.12799104
('Episode ', 7, 'Starts!')
('Collision!', 0)
-42.1048655251
('Episode ', 8, 'Starts!')
('Collision!', 0)
-39.2738570896
('Episode ', 9, 'Starts!')
('Collision!', 0)       
-10824.5362462
('Episode ', 10, 'Starts!')
('Collision!', 0)
-23.8398000912
('Episode ', 11, 'Starts!')
('Collision!', 0)
-47.317066575
('Episode ', 12, 'Starts!')
('Collision!', 0)                  
-43901.019254
('Episode ', 13, 'Starts!')
('Collision!', 0)         
-12836.2141185
('Episode ', 14, 'Starts!')
('Collision!', 0)
-62.8373894162
('Episode ', 15, 'Starts!')
('Collision!', 0)
-81.9825942425
('Episo

KeyboardInterrupt: 

[[0.15473302] [0.449626  ]]
    
[[0.34095567] [0.54793298]]

Converged: 6
Diverged: 8
Local Minima: 13